# Run PEST and Parallel PEST
### This should be super easy ~

In [1]:
import os
import shutil
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt;
import psutil
import sys
import pyemu
import flopy
import swatp_pst
assert "dependencies" in flopy.__file__
assert "dependencies" in pyemu.__file__
assert "dependencies" in swatp_pst.__file__

# 01. Specify a path to the model folder

In [2]:
# path to project directory
prj_dir = "D:\\spark\\jj"

In [3]:
main_opt_path = os.path.join(prj_dir, 'main_opt')
os.chdir(main_opt_path)

In [4]:
os.getcwd()

'D:\\spark\\jj\\main_opt'

# 02. Initial run for adjusting weights

In [5]:
# PEST control file name
pst_name = "swatp_test.pst"

In [6]:
# initial run
pyemu.os_utils.run(f'pestpp-glm.exe {pst_name}' , cwd=".")

## 02-01 Change weights to make all of observation data visible 

In [6]:
# reweight
pst = pyemu.Pst(pst_name)
pst.phi

2033.2332590533408

In [ ]:
# you can assign any values to balanced phi value for each group
# balanced_groups = {grp:1000 for grp in pst.nnz_obs_groups}
# pst.adjust_weights(obsgrp_dict=balanced_groups)

In [7]:
# Let's create a new control file with the number of iterations set to 30 and incorporate reweighted factors.
pst.control_data.noptmax = 30
pst.write(os.path.join(main_opt_path,'swatp_sen.pst'), version=2)

noptmax:30, npar_adj:5, nnz_obs:124


# 03. Perform a history matching in parallel processing

In [5]:
# check the number of available cores
num_workers = psutil.cpu_count(logical=False)
num_workers

96

In [10]:
# set a path to the main directory of model
m_d = os.path.join(prj_dir, "swatp_nw_glm")

In [ ]:
os.chdir(prj_dir) # move the loction to project directory
# run PEST in parallel
pyemu.os_utils.start_workers(main_opt_path,"pestpp-glm",'swatp_test_nweight.pst',
                            num_workers=num_workers,
                            worker_root='.',
                            master_dir=m_d)

In [6]:
# check number of cores on your computer
num_workers = psutil.cpu_count(logical=False)

In [7]:
pst_ies = pyemu.Pst(os.path.join(main_opt_path,"swatp_test_nweight.pst"))
# set IES
pst_ies.pestpp_options['ies_num_reals'] = 100 # number of realization
pst_ies.control_data.noptmax = 10 # number of iteration
pst_ies.write('swatp_nw_ies.pst', version=2) # write new IES control file

noptmax:10, npar_adj:5, nnz_obs:124


In [7]:
os.chdir(os.pardir)

In [8]:
os.getcwd()

'D:\\spark\\jj'

In [9]:
m_d = os.path.join(os.getcwd(), "swatp_nw_ies")

In [10]:
main_opt_path

'D:\\spark\\jj\\main_opt'

In [12]:
pyemu.os_utils.start_workers(main_opt_path, # the folder which contains the "template" PEST dataset
                            'pestpp-ies', #the PEST software version we want to run
                            'swatp_nw_ies.pst', # the control file to use with PEST
                            num_workers=num_workers, #how many agents to deploy
                            worker_root='.', #where to deploy the agent directories; relative to where python is running
                            master_dir=m_d, #the manager directory,
                            # reuse_master=True
                            )

Exception: start_workers() master returned non-zero: 1

# 04. Let's see how we did:

In [ ]:
os.chdir(m_d)
pst = pyemu.Pst('mb_zon_rw.pst')
pst.phi

In [ ]:
pst.res.loc[pst.nnz_obs_names,:]

In [ ]:
# plot 1 to 1
figs = pst.plot(kind="1to1")

In [ ]:
# plot phi progress
pyemu.plot_utils.phi_progress(pst)